In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os

# --------------------------------------------------
# 1. Pré-processamento de Dados
# --------------------------------------------------

# Certifique-se de que o arquivo de texto existe
file_path = 'texto.txt'
if not os.path.exists(file_path):
    print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    print("Certifique-se de que o arquivo está no mesmo diretório do script.")
    exit()

# Carrega o texto do arquivo
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Cria o vocabulário (mapeamento de caracteres para inteiros)
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
n_vocab = len(chars)
n_chars = len(text)

# Cria as sequências de treinamento
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

# --------------------------------------------------
# 2. Modelo RNN (LSTM)
# --------------------------------------------------

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        
        # Correção principal: Pega a saída apenas do último passo de tempo
        output = self.fc(output[:, -1, :])
        
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size),
                torch.zeros(self.n_layers, batch_size, self.hidden_size))

# --------------------------------------------------
# 3. Preparação para o Treino
# --------------------------------------------------

class TextDataset(Dataset):
    def __init__(self, dataX, dataY):
        self.dataX = torch.tensor(dataX, dtype=torch.long)
        self.dataY = torch.tensor(dataY, dtype=torch.long)
    
    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, idx):
        return self.dataX[idx], self.dataY[idx]

dataset = TextDataset(dataX, dataY)
batch_size = 128 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instancia o modelo, a função de perda e o otimizador
model = CharRNN(n_vocab, hidden_size=256, output_size=n_vocab, n_layers=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Define o número de épocas para o treinamento
num_epochs = 5

# --------------------------------------------------
# 4. Treinamento do Modelo
# --------------------------------------------------

print("Iniciando o treinamento do modelo...")
for epoch in range(num_epochs):
    # O hidden state é inicializado DENTRO do loop de batches
    for inputs, labels in dataloader:
        
        # AQUI ESTÁ A CORREÇÃO PRINCIPAL:
        # A nova batch_size é obtida do tamanho do lote atual de inputs.
        batch_size = inputs.size(0)
        hidden = model.init_hidden(batch_size)

        optimizer.zero_grad()
        
        # Desconecta os estados ocultos
        hidden = tuple([h.data for h in hidden])
        
        output, hidden = model(inputs, hidden)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
    
    print(f"Época [{epoch+1}/{num_epochs}], Perda: {loss.item():.4f}")

print("Treinamento concluído.")

# --------------------------------------------------
# 5. Geração de Texto
# --------------------------------------------------

def generate_text(model, start_string, length):
    # Converte a string de início para um tensor
    input_eval = torch.tensor([char_to_int[s] for s in start_string]).unsqueeze(0)
    
    text_generated = start_string
    model.eval() # Coloca o modelo em modo de avaliação
    
    with torch.no_grad():
        hidden = model.init_hidden(1)
        for _ in range(length):
            output, hidden = model(input_eval, hidden)
            
            # Obtém o caractere previsto
            predicted_char = int_to_char[torch.argmax(output[-1]).item()]
            
            # Adiciona ao texto gerado
            text_generated += predicted_char
            
            # Atualiza a entrada para a próxima etapa
            input_eval = torch.tensor([[char_to_int[predicted_char]]])
    
    return text_generated


In [ ]:

# Exemplo de uso:
print("\n--- Texto Gerado ---")
# Use uma string de início que faça sentido com o estilo do autor
start_text = "Format                      : Matroska Format  : Version 4 / Vers" 
generated_text = generate_text(model, start_string=start_text, length=100)
print(generated_text)


--- Texto Gerado ---
Format                      : Matroska Format  : Version 4 / Version                                                                                                 
